In [62]:
!pip install tqdm

In [64]:
import requests as r
import pandas as pd
from fuzzywuzzy import fuzz
from functools import cache
from tqdm import tqdm

In [49]:
    def stocks():

        raw_symbols = r.get(
            f"https://cloud.iexapis.com/stable/ref-data/symbols?token=WOOOPS"
        ).json()
        symbols = pd.DataFrame(data=raw_symbols)

        symbols["description"] = "$" + symbols["symbol"] + ": " + symbols["name"]
        symbols["id"] = symbols["symbol"]

        symbols = symbols[["id", "symbol", "name", "description"]]

        return symbols



    def coins():

        raw_symbols = r.get("https://api.coingecko.com/api/v3/coins/list").json()
        symbols = pd.DataFrame(data=raw_symbols)

        symbols["description"] = "$$" + symbols["symbol"] + ": " + symbols["name"]
        symbols = symbols[["id", "symbol", "name", "description"]]

        return symbols

In [51]:
df = pd.concat([stocks(), coins()])
df

,id,symbol,name,description
0,A,A,Agilent Technologies Inc.,$A: Agilent Technologies Inc.
1,AA,AA,Alcoa Corp,$AA: Alcoa Corp
2,AAA,AAA,Listed Funds Trust - AAF First Priority CLO Bo...,$AAA: Listed Funds Trust - AAF First Priority ...
3,AAAU,AAAU,Goldman Sachs Physical Gold ETF Shares - Goldm...,$AAAU: Goldman Sachs Physical Gold ETF Shares ...
4,AAC,AAC,Ares Acquisition Corporation - Class A,$AAC: Ares Acquisition Corporation - Class A
...,...,...,...,...
6565,zyro,zyro,Zyro,$$zyro: Zyro
6566,zytara-dollar,zusd,Zytara Dollar,$$zusd: Zytara Dollar
6567,zyx,zyx,ZYX,$$zyx: ZYX
6568,zzz-finance,zzz,zzz.finance,$$zzz: zzz.finance


In [79]:
    def search_symbols(search: str):
        """Performs a fuzzy search to find stock symbols closest to a search term.

        Parameters
        ----------
        search : str
            String used to search, could be a company name or something close to the companies stock ticker.

        Returns
        -------
        List[tuple[str, str]]
            A list tuples of every stock sorted in order of how well they match. Each tuple contains: (Symbol, Issue Name).
        """

        try:
            if search_index[search]: return search_index[search]
        except KeyError:
            pass



        search = search.lower()

        df["Match"] = df.apply(
            lambda x: fuzz.ratio(search, f"{x['symbol']}".lower()),
            axis=1,
        )

        df.sort_values(by="Match", ascending=False, inplace=True)
        if df["Match"].head().sum() < 300:
            df["Match"] = df.apply(
                lambda x: fuzz.partial_ratio(search, x["name"].lower()),
                axis=1,
            )

            df.sort_values(by="Match", ascending=False, inplace=True)

        symbols = df.head(20)
        symbol_list = list(zip(list(symbols["symbol"]), list(symbols["description"])))
        
        return symbol_list

In [91]:
search_list = df['id'].to_list() + df['description'].to_list()
search_index = {}

In [92]:

for i in tqdm(search_list):
    search_index[i] = search_symbols(i)

  5%|▍         | 1545/33892 [06:51<2:23:40,  3.75it/s]


KeyboardInterrupt: 

In [89]:
import pickle



with open('search_index.pickle', 'wb') as handle:
    pickle.dump(search_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)
